## Data preprocessing
#### Nov.9.2022

In [2]:
# capture
import pandas as pd
import numpy as np

df_dietary = pd.read_sas('./dataset/P_DSQTOT.XPT')
df_nutrition = pd.read_sas('./dataset/P_DR1TOT.XPT')
df_dental = pd.read_sas('./dataset/P_OHXDEN.XPT')

/home/changliu020/anaconda3/envs/ece143/lib/python3.10/site-packages/pandas/io/sas/sas_xport.py:475: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[x] = v


In [3]:
df_data = pd.merge(df_dietary, df_nutrition, on='SEQN')
df_data = pd.merge(df_data, df_dental, on='SEQN')
df_data.info()
df_data.head()
# df_data.to_csv('train_data.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13772 entries, 0 to 13771
Columns: 375 entries, SEQN to OHX31SE
dtypes: float64(245), object(130)
memory usage: 39.5+ MB


,SEQN,DSDCOUNT,DSDANCNT,DSD010,DSD010AN,DSQTKCAL,DSQTPROT,DSQTCARB,DSQTSUGR,DSQTFIBE,...,OHX14SE,OHX15SE,OHX18SE,OHX19SE,OHX20SE,OHX21SE,OHX28SE,OHX29SE,OHX30SE,OHX31SE
0,109263.0,2.000000e+00,5.397605e-79,1.0,2.0,18.0,NaN,3.5,3.0,NaN,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
1,109264.0,5.397605e-79,5.397605e-79,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
2,109265.0,5.397605e-79,5.397605e-79,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
3,109266.0,5.397605e-79,5.397605e-79,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
4,109269.0,5.397605e-79,5.397605e-79,2.0,2.0,NaN,NaN,NaN,NaN,NaN,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''


In [4]:
df_dropna = df_data.dropna()
df_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Columns: 375 entries, SEQN to OHX31SE
dtypes: float64(245), object(130)
memory usage: 0.0+ bytes


#### Summary: Our dataset has 13772 observations and 374 features with a lot of empty entries. If we drop na, there is simply nothing left.

In [5]:
attr_dict = {}    # { index: attribute meaning }
with open('./dataset/attributes_meaning.txt', 'r') as f:
    for line in f.readlines():
        idx, attr = line.split(' - ')
        attr_dict[idx] = attr.strip()

In [6]:
len(attr_dict)

375

In [7]:
# Explore NA of dietary
for attr in df_data:
    if attr[:2] == 'DS' and df_data[attr].isna().sum() > len(df_data) / 2:
        print(attr, attr_dict[attr])

DSQTKCAL Energy (kcal)
DSQTPROT Protein (gm)
DSQTCARB Carbohydrate (gm)
DSQTSUGR Total sugars (gm)
DSQTFIBE Dietary fiber (gm)
DSQTTFAT Total fat (gm)
DSQTSFAT Total saturated fatty acids (gm)
DSQTMFAT Total monounsaturated fatty acids (gm)
DSQTPFAT Total polyunsaturated fatty acids (gm)
DSQTCHOL Cholesterol (mg)
DSQTLYCO Lycopene (mcg)
DSQTLZ Lutein + zeaxanthin (mcg)
DSQTVB1 Thiamin (Vitamin B1) (mg)
DSQTVB2 Riboflavin (Vitamin B2) (mg)
DSQTNIAC Niacin (mg)
DSQTVB6 Vitamin B6 (mg)
DSQTFA Folic acid (mcg)
DSQTFDFE Folate, DFE (mcg)
DSQTCHL Total choline (mg)
DSQTVB12 Vitamin B12 (mcg)
DSQTVC Vitamin C (mg)
DSQTVK Vitamin K (mcg)
DSQTVD Vitamin D (D2 + D3) (mcg)
DSQTCALC Calcium (mg)
DSQTPHOS Phosphorus (mg)
DSQTMAGN Magnesium (mg)
DSQTIRON Iron (mg)
DSQTZINC Zinc (mg)
DSQTCOPP Copper (mg)
DSQTSODI Sodium (mg)
DSQTPOTA Potassium (mg)
DSQTSELE Selenium (mcg)
DSQTCAFF Caffeine (mg)
DSQTIODI Iodine (mcg)


For dietary data, most of the missing data is about the precise quantitative data, e.g. energy by kcal, protein by gm. 

This is reasonable for the normal survey participant, who typically does not measure nutrient intake in precise quantitative terms.

Suggestion: drop these attributes.

In [8]:
# Explore NA of nutrition
for attr in df_data:
    if attr[:2] in ('DR', 'WT') and df_data[attr].isna().sum() > len(df_data) / 2:
        print(attr, attr_dict[attr])

DR1SKY Type of salt used yesterday
DRQSDT1 Weight loss/Low calorie diet
DRQSDT2 Low fat/Low cholesterol diet
DRQSDT3 Low salt/Low sodium diet
DRQSDT4 Sugar free/Low sugar diet
DRQSDT5 Low fiber diet
DRQSDT6 High fiber diet
DRQSDT7 Diabetic diet
DRQSDT8 Weight gain/Muscle building diet
DRQSDT9 Low carbohydrate diet
DRQSDT10 High protein diet
DRQSDT11 Gluten-free/Celiac diet
DRQSDT12 Renal/Kidney diet
DRQSDT91 Other special diet
DRD350A Clams eaten during past 30 days
DRD350AQ # of times clams eaten in past 30 days
DRD350B Crabs eaten during past 30 days
DRD350BQ # of times crabs eaten in past 30 days
DRD350C Crayfish eaten during past 30 days
DRD350CQ # of times crayfish eaten past 30 days
DRD350D Lobsters eaten during past 30 days
DRD350DQ # of times lobsters eaten past 30 days
DRD350E Mussels eaten during past 30 days
DRD350EQ # of times mussels eaten in past 30 days
DRD350F Oysters eaten during past 30 days
DRD350FQ # of times oysters eaten in past 30 days
DRD350G Scallops eaten duri

For nutrition data, most of the missing data are about special diets such as "high protein diet" and times of eating specific food during 30 days.

Suggestion: treat them as 0, since it makes sense that normal participants never eat such kind of food.

In [9]:
# Explore NA of oral health
for attr in df_data:
    if attr[:2] == 'OH' and df_data[attr].isna().sum() > len(df_data) / 2:
        print(attr, attr_dict[attr])

In [10]:
for attr in df_dental:
    if df_dental[attr].isna().sum() > 0:
        print(attr, df_dental[attr].isna().sum(), attr_dict[attr])

OHXIMP 4024 Dental Implant: yes / no?
OHX01TC 494 Tooth Count: #1
OHX02TC 494 Tooth Count: #2
OHX03TC 494 Tooth Count: #3
OHX04TC 494 Tooth Count: #4
OHX05TC 494 Tooth Count: #5
OHX06TC 494 Tooth Count: #6
OHX07TC 494 Tooth Count: #7
OHX08TC 494 Tooth Count: #8
OHX09TC 494 Tooth Count: #9
OHX10TC 494 Tooth Count: #10
OHX11TC 494 Tooth Count: #11
OHX12TC 494 Tooth Count: #12
OHX13TC 494 Tooth Count: #13
OHX14TC 494 Tooth Count: #14
OHX15TC 494 Tooth Count: #15
OHX16TC 494 Tooth Count: #16
OHX17TC 494 Tooth Count: #17
OHX18TC 494 Tooth Count: #18
OHX19TC 494 Tooth Count: #19
OHX20TC 494 Tooth Count: #20
OHX21TC 494 Tooth Count: #21
OHX22TC 494 Tooth Count: #22
OHX23TC 494 Tooth Count: #23
OHX24TC 494 Tooth Count: #24
OHX25TC 494 Tooth Count: #25
OHX26TC 494 Tooth Count: #26
OHX27TC 494 Tooth Count: #27
OHX28TC 494 Tooth Count: #28
OHX29TC 494 Tooth Count: #29
OHX30TC 494 Tooth Count: #30
OHX31TC 494 Tooth Count: #31
OHX32TC 494 Tooth Count: #32
OHXRCAR 5764 Root caries
OHXRCARO 5764 Othe

There are 494 entries marked as didn't complete exam, and most of the attributes in such entries are now, so we can drop them.

In [11]:
df_dental = df_dental[df_dental['OHDEXSTS'] != 3]

In [12]:
df_dental

,SEQN,OHDEXSTS,OHDDESTS,OHXIMP,OHX01TC,OHX02TC,OHX03TC,OHX04TC,OHX05TC,OHX06TC,...,OHX14SE,OHX15SE,OHX18SE,OHX19SE,OHX20SE,OHX21SE,OHX28SE,OHX29SE,OHX30SE,OHX31SE
0,109263.0,1.0,1.0,NaN,4.0,4.0,4.0,1.0,1.0,1.0,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
1,109264.0,1.0,1.0,2.0,4.0,2.0,2.0,2.0,2.0,2.0,...,b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0',b'0'
2,109265.0,1.0,1.0,NaN,4.0,4.0,4.0,1.0,1.0,1.0,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
3,109266.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
4,109269.0,1.0,1.0,NaN,4.0,4.0,4.0,1.0,1.0,1.0,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13766,124817.0,1.0,1.0,2.0,4.0,2.0,2.0,2.0,2.0,2.0,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
13767,124818.0,1.0,1.0,2.0,4.0,2.0,2.0,4.0,2.0,2.0,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
13768,124819.0,1.0,1.0,NaN,4.0,4.0,4.0,1.0,1.0,1.0,...,b'',b'',b'',b'',b'',b'',b'',b'',b'',b''
13769,124820.0,1.0,1.0,NaN,4.0,4.0,4.0,1.0,1.0,1.0,...,b'9',b'9',b'9',b'0',b'0',b'0',b'0',b'0',b'0',b'9'


"OHX02SE" - "OHX31SE" are about dental sealants, where 9384/13772 entries miss these attributes. Suggest to drop these attributes.

# In conclusion, preliminary data processing will include
- Drop "DSQTKCAL - DSQTIODI"
- Set "DR1SKY - DRD370UQ" to 0
- Drop "OHX02SE" - "OHX31SE"